In [1]:
TOP_N = 50

In [ ]:
import sqlite3

try:
    conn = sqlite3.Connection('data/data.db')
    cursor = conn.cursor()

    statements = [
        "drop table if exists chunks",
        "drop table if exists articles",
        "drop table if exists embeddings",

        """
        create table if not exists embeddings (
            id integer primary key,
            vector blob,
            article_id integer,
            chunk_id integer,
            model text
        )
        """,

        """
        create table if not exists chunks (
            chunk_id integer,
            chunk text,
            article_id integer
        )
        """,
        
        """
        create table if not exists articles (
            article_id integer,
            title text,
            clean_title text,
            url text,
            count integer
        )
        """
    ]

    for statement in statements:
        cursor.execute(statement)
        conn.commit()

finally:
    conn.close()

In [ ]:
# import sqlite3
# import json 
# import base64

# insert_statement = "insert into embeddings (vector, article_id, chunk_id, model) values (?, ?, ?, ?)"

# def insert_embeddings(path, insert_statement):
#     try:
#         conn = sqlite3.Connection('data/data.db')
#         cursor = conn.cursor()

#         buffer = []
#         with open(path, 'r') as file:
#             for line in file:
#                 jsonl = json.loads(line)
#                 jsonl['embedding'] = base64.b64decode(jsonl.get('embedding', ''))
#                 buffer.append((
#                     jsonl.get('embedding', ''), 
#                     jsonl.get('article_id', -1), 
#                     jsonl.get('chunk_id', -1), 
#                     jsonl.get('model', '')
#                 ))
#                 if len(buffer) > 10000:
#                     print("Clearing buffer.")
#                     cursor.executemany(insert_statement, buffer)
#                     conn.commit()
#                     buffer = []
#             if buffer:
#                 print("Clearing buffer.")
#                 cursor.executemany(insert_statement, buffer)
#                 conn.commit()

#     finally:
#         conn.close()


# insert_embeddings('data/embeddings2.jsonl', insert_statement)
# insert_embeddings('data/embeddings3.jsonl', insert_statement)

In [4]:
import pandas as pd

conn = sqlite3.Connection('data/data.db')

pd.read_parquet('data/articles_top5_len40_stride15.parquet') \
    .rename(columns={'lower_title': 'clean_title'}) \
    .to_sql('articles', conn, if_exists='append', index=False)

conn.commit()
conn.close()

In [ ]:
import pandas as pd
import json 

conn = sqlite3.Connection('data/data.db')

buffer = []
with open('data/chunks_top5_len40_stride15.jsonl', 'r') as file:
    for line in file:
        buffer.append(json.loads(line))

        if len(buffer) > 100000:
            pd.DataFrame(buffer).to_sql('chunks', conn, if_exists='append', index=False)
            buffer = []
            conn.commit()

if buffer:
    pd.DataFrame(buffer).to_sql('chunks', conn, if_exists='append', index=False)
    buffer = []
    conn.commit()

conn.close()

In [6]:
import pandas as pd

conn = sqlite3.Connection('data/data.db')

indices = [
    'create index idx_articles_article_id on articles(article_id)',
    'create index idx_articles_count on articles(count)',
    'create index idx_chunks_article_id on chunks(article_id)',
    'create index idx_chunks_chunk_id on chunks(chunk_id)',
    # 'create index idx_embeddings_article_id on embeddings(article_id)',
    # 'create index idx_embeddings_chunk_id on embeddings(chunk_id)'
]

for index in indices:
    conn.execute(index)
    conn.commit()

conn.close()